In [1]:
# Imports
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import random

In [2]:
# Define neural network structure
global nn_structure,total_unknowns,act_fn,x,y

nn_structure = [2,3,1]                                         # Number of nodes in layers :[Input_layer, hidden_layer1, output_layer]
act_fn = ['logsig','logsig','logsig']                          # Activation function ; size should be same as nn_structure
x = np.array([[0,0],[0,1],[1,0],[1,1]])                        # Input to neural network
y = np.array([0,1,1,1])                                        # Output of neural network - OR Gate

total_unknowns = 0
for i in range(1,len(nn_structure)):
    total_unknowns += (nn_structure[i]*nn_structure[i-1])      # adding weights of 'i'th layer
    total_unknowns += nn_structure[i]                          # adding biases of 'i'th layer

In [3]:
# Activation functions
def log_sig(x):
    return 1/(1+np.exp(-x))

def linear(x):
    return x

In [4]:
#Objective function

def error(nn_params):   
    pointer = 0
    weights = []
    biases = []
    
    # Extract layerwise weights and biases from particle position(nn_param)
    for i in range(1,len(nn_structure)):
        w = np.array(nn_params[pointer:(pointer+nn_structure[i]*nn_structure[i-1])]).reshape(nn_structure[i],nn_structure[i-1])
        weights.append(w)
        pointer += (nn_structure[i]*nn_structure[i-1])
        
        b = np.array(nn_params[pointer:(pointer+nn_structure[i])]).reshape(nn_structure[i],1)
        biases.append(b)
        pointer += nn_structure[i]
    
    # Predict output variable
    y_pred = []
    for inp in x:
        prev_layer = inp.reshape(1,2).T
        for layer in range(len(weights)):
            prev_layer = weights[layer]@prev_layer + biases[layer]
            activation = act_fn[layer]
            if activation == 'logsig':
                prev_layer = log_sig(prev_layer)
            elif activation == 'linear':
                prev_layer = linear(prev_layer)
        y_pred.append(prev_layer[0][0])
    
    # Calculate error value. We can use RMSE / absolute mean error. 
    rmse = mean_squared_error(y_pred,y.flatten())    
    return 10*rmse

In [5]:
class Particle:
    def __init__(self):
        self.position = np.random.rand(total_unknowns) #We initialize position randomly in 6(w1) + 3(b1) + 3(w2) + 1(b2) = 13 dimension space
        self.velocity = np.random.rand(total_unknowns) #Initialize velocity randomly
        self.cost = error(self.position)               #Calculate cost value associated with position of particle
        self.best_position = self.position             #Initialize local best as current position
        self.best_cost = 100000000                     #Initialize best cost

class GlobalBest:                                      #Initialize global best
    def __init__(self):
        self.position = np.random.rand(13)
        self.cost = 100000000

In [6]:
# PSO param initialization
swarm_size = 10
iteration = 15
list_of_particles = []
global_best = GlobalBest()

for i in range(swarm_size):
    particle = Particle()
    if particle.cost < global_best.cost:
        global_best.cost = particle.cost
        global_best.position = particle.position  
    list_of_particles.append(particle)

In [7]:
# Main PSO
w = 0.3
c1 = 0.50
c2 = 0.64
for itr in range(iteration):
    #print("Iteration :",itr)
    for particle in list_of_particles:
        #Calculate new velocity
        newVelocity = (w*particle.velocity) + (round(c1*(-3 + 6*random.random()))*(particle.best_position - particle.position)) + (round(c2*(-3 + 6*random.random()))*(global_best.position - particle.position))

        #Calculate new position
        newPosition = particle.position + newVelocity

        current_cost = error(particle.position)
        new_cost = error(newPosition)

        if new_cost < current_cost:
            # Update velocity,position,cost
            particle.velocity = newVelocity
            particle.position = newPosition
            particle.cost = new_cost

            # Upodate local best
            if particle.cost < particle.best_cost:
                particle.best_position = particle.position;
                particle.best_cost = particle.cost;

                #Update global best
                if particle.best_cost < global_best.cost:
                    global_best.cost = particle.best_cost
                    global_best.position = particle.position                  

In [8]:
print(global_best.cost)

7.958213330234088e-05


In [9]:
# predict output using trained neural network

x = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([0,1,1,1])
nn_params = global_best.position
pointer = 0
weights = []
biases = []
for i in range(1,len(nn_structure)):
    w = np.array(nn_params[pointer:(pointer+nn_structure[i]*nn_structure[i-1])]).reshape(nn_structure[i],nn_structure[i-1])
    weights.append(w)
    pointer += (nn_structure[i]*nn_structure[i-1])

    b = np.array(nn_params[pointer:(pointer+nn_structure[i])]).reshape(nn_structure[i],1)
    biases.append(b)
    pointer += nn_structure[i]

y_pred = []
for inp in x:
    prev_layer = inp.reshape(1,2).T
    for layer in range(len(weights)):
        prev_layer = weights[layer]@prev_layer + biases[layer]
        activation = act_fn[layer]
        if activation == 'logsig':
            prev_layer = log_sig(prev_layer)
        elif activation == 'linear':
            prev_layer = linear(prev_layer)
    y_pred.append(prev_layer[0][0])
print("Predicted output :",y_pred)
print("Actual output :",y)

Predicted output : [2.4265442176783237e-14, 0.9967426117622791, 0.9967424400146043, 0.9967426117622791]
Actual output : [0 1 1 1]
